In [1]:
import pandas as pd
import torch

#specter='SPECTER1'
specter='SPECTER2'
#modele='DeepSeek'
modele='OLMO3.1_32B'
#compute_method='avg'
compute_method='max'



papers=pd.read_json('submission_metadata.json').T
papers.index.name = 'row_id'

malicious=pd.read_json('101ranking.json')
authors=pd.read_json('authors92K.json')


reecrit_total=pd.read_csv(modele+'/reecrit_TOTAL.csv')
reecrit_total['row_id']=list(papers.index)


papers_tensor=torch.load(specter+'/Papers'+specter+'.pt')
malicious_tensor=torch.load(specter+'/malicious'+specter+'.pt')
authors_tensor=torch.load(specter+'/authors'+specter+'.pt')
rewritten_tensor=torch.load(modele+'/'+specter+'/rewritten.pt')

rewritten_all_papers=torch.load(modele+'/'+specter+'/rewritten_TOTAL.pt')
reecrit_total_tensor=rewritten_all_papers

print(authors_tensor.shape)
print(papers_tensor.shape)
print(malicious_tensor.shape)
print(rewritten_tensor.shape)
print(rewritten_all_papers.shape)
#reecrit_total
#malicious


torch.Size([92819, 768])
torch.Size([3218, 768])
torch.Size([300, 768])
torch.Size([300, 768])
torch.Size([3218, 768])


In [2]:
from torch.nn.functional import cosine_similarity

def my_cosine_similarity(paper,list_of_papers): 
    """
    Calcul la similarité cosinus entre un papier et une les papiers d'un auteur
    """
    rez=[]
    for paper2 in list_of_papers:
        rez.append(cosine_similarity(paper,paper2,dim=0))
    return rez

def get_similarity(author, paper, mode='max'):
        similarities = my_cosine_similarity(paper,authors_dico[author])
        if mode== "max":
            return max(similarities)
        elif mode == "avg":
            return torch.stack(similarities).mean(dim=0)


In [3]:
i=0
authors_dico={}
for index,row in authors.iterrows():
    if(row['AuthorID'] not in authors_dico):
        authors_dico[row['AuthorID']]=[] #si l'auteur est pas dans le dico, on lui crée une liste vide
    authors_dico[row['AuthorID']].append(authors_tensor[i]) 
    i+=1

targets=[]
for elt in malicious['colluding_reviewer_profile_link']:
    targets.append(elt[34:])
malicious['target'] = targets

liste_ids=[]
for id, row in malicious.iterrows():
  for index,d in papers.iterrows():
    if(d['title']==row['title']):
      liste_ids.append(index)

malicious['id']=liste_ids

In [4]:
#il ne faut garder que les bons auteurs ! ceux qui ont été des targets.
good_authors={}
i=0
for id,elt in malicious.iterrows():
    good_authors[elt['target']] = authors_dico[elt['target']]

In [ ]:
# Calcul des rankings originaux - À EXÉCUTER UNE FOIS PAR SPECTER
# Ce code génère Original_rankings_SPECTER1.json ou Original_rankings_SPECTER2.json
# selon la valeur de la variable 'specter' définie en Cell 0

import os

original_rankings_file = "Rankings/Original_rankings_"+specter+".json"

# Vérifier si le fichier existe déjà pour ce SPECTER
if not os.path.exists(original_rankings_file):
    print(f"Génération de {original_rankings_file}...")
    dico_rankings_original={}
    i=0
    for author_id,author_tens in good_authors.items():
        dico_rankings_original[author_id]={}
        for paper_tens,paper_metadata in zip(papers_tensor,papers.iterrows()):
            dico_rankings_original[author_id][paper_metadata[0]]=get_similarity(author_id,paper_tens,mode=compute_method)
        i+=1
        if(i%10==0):
            print(i)
    
    # Convertir tensors en floats pour JSON
    for key,value in dico_rankings_original.items():
        for author_id,author_tens in value.items():
            value[author_id]=float(author_tens)
    
    import json
    with open(original_rankings_file, "w") as f:
        json.dump(dico_rankings_original, f)
    print(f"✅ {original_rankings_file} généré avec succès!")
else:
    print(f"ℹ️ {original_rankings_file} existe déjà. Supprimez-le pour le régénérer.")

In [6]:
dico_rankings={}
i=0
for author_id,author_tens in good_authors.items():
    dico_rankings[author_id]={}
    for paper_tens,paper_metadata in zip(reecrit_total_tensor,reecrit_total.iterrows()):
        dico_rankings[author_id][paper_metadata[1]['row_id']]=get_similarity(author_id,paper_tens,mode=compute_method)
    i+=1
    if(i%10==0):
        print(i)


10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280


In [7]:
for key,value in dico_rankings.items():
    for author_id,author_tens in value.items():
        value[author_id]=float(author_tens) #transforme les tensors en floats psk c'est pas serialisable pour en faire un JSON
import json

# with open("Rankings/Original_rankings.json", "w") as f: #faut être sûr l'ami.
#     #soit quali dans l'organisation.
#     json.dump(dico_rankings, f)

with open("Rankings/"+modele+specter+"rewritten_rankings.json", "w") as f: #faut être sûr l'ami.
    #soit quali dans l'organisation.
    json.dump(dico_rankings, f)

In [ ]:
import json
with open("Rankings/Original_rankings.json", "r") as f:
   my_dict = json.load(f)
with open("Rankings/"+modele+specter+"rewritten_rankings.json", "r") as f:
    my_dict2 = json.load(f)




In [9]:
print(my_dict['~Adam_W_Harley1'])
print(my_dict2['~Adam_W_Harley1'])

{'zyhxRc9bew': 0.7557873725891113, 'zyZkaqNnpa': 0.8191495537757874, 'zuXyQsXVLF': 0.7254528999328613, 'ztDxO15N7f': 0.7328059673309326, 'zsOOqjaj2z': 0.6544883847236633, 'zrUEHZ6s9C': 0.7568599581718445, 'zrLxHYvIFL': 0.7821409106254578, 'zrCmeqV3Sz': 0.7452417612075806, 'zqyVjCjhYD': 0.7091138958930969, 'zqOcW3R9rd': 0.6574876308441162, 'zq4vFneRiA': 0.7455849051475525, 'zpVCITHknd': 0.6886884570121765, 'znudaK78u8': 0.8165709972381592, 'znY173SCxu': 0.7427858114242554, 'znW5jNIOED': 0.7219229340553284, 'zn5ihqknGj': 0.7139936685562134, 'zmWNe1V6jg': 0.6956902146339417, 'zkfyOkBVpz': 0.7852134704589844, 'zjpjsJeVJZ': 0.7792863845825195, 'zfHCKDzzC8': 0.6899831891059875, 'zfCNwRQ569': 0.6884727478027344, 'zdli6OxpWd': 0.7312005758285522, 'zaQ7wV9NOg': 0.7200934290885925, 'zXckveawHa': 0.7938082814216614, 'zWxKYyW9ik': 0.743645191192627, 'zW1uVN6Mbv': 0.7431675791740417, 'zUYfbdNl1m': 0.7000747919082642, 'zTSlm4nmlH': 0.7848570942878723, 'zQTi3pziFp': 0.7625412940979004, 'zQOYGDc9pu': 

In [10]:
for id,row in malicious.iterrows():
    my_dict[row['target']]['ORIGINAL']=my_dict[row['target']].pop(row['id'])

#for key,value in my_dict.items(): #On a remplacé cible ID par ORIGINAL
    #print(value['ORIGINAL'])

In [11]:
def find_name_rank(name,liste):
    for i,elt in enumerate(liste):
        if(name==elt[0]): 
            return i

In [12]:
id=0
for id,row in malicious.iterrows():
    similarity_score=get_similarity(row['target'],malicious_tensor[id],mode=compute_method)
    my_dict[row['target']]['MALICIOUS']=similarity_score
    similarity_score_rewritten=get_similarity(row['target'],rewritten_tensor[id],mode=compute_method)
    my_dict[row['target']]['REWRITTEN']=similarity_score_rewritten

In [13]:
dict_sorted={}
dict_sorted2={}
for key,value in my_dict.items():
    dict_sorted[key]=sorted(value.items(), key=lambda x: x[1], reverse=True)

for key,value in my_dict2.items():
    dict_sorted2[key]=sorted(value.items(), key=lambda x: x[1], reverse=True)

In [14]:
moyenne_ORIGINAL=[]
moyenne_MALICIOUS=[]
moyenne_REWRITTEN=[]

for id,row in malicious.iterrows():
    a=find_name_rank('ORIGINAL',dict_sorted[row['target']])
    b=find_name_rank('MALICIOUS',dict_sorted[row['target']])
    c=find_name_rank('REWRITTEN',dict_sorted[row['target']])
    
    moyenne_ORIGINAL.append(a)
    moyenne_MALICIOUS.append(b)
    moyenne_REWRITTEN.append(c)


In [15]:
compteur=0
for i in range(0,len(moyenne_MALICIOUS)):
    if(moyenne_MALICIOUS[i]<=5):
        compteur+=1
print(compteur)

283


In [17]:
compteur=0
for i in range(0,len(moyenne_REWRITTEN)):
    if(moyenne_REWRITTEN[i]<=5):
        compteur+=1
print(compteur)

203


In [18]:
for a,b,c in zip(moyenne_ORIGINAL,moyenne_MALICIOUS,moyenne_REWRITTEN):
    pass
    #print(a,b,c)

ORIGINAL_MEAN=sum(moyenne_ORIGINAL)/len(moyenne_ORIGINAL)
MALICIOUS_MEAN=sum(moyenne_MALICIOUS)/len(moyenne_MALICIOUS)
REWRITTEN_MEAN=sum(moyenne_REWRITTEN)/len(moyenne_REWRITTEN)

In [19]:
ORIGINAL_MEAN-MALICIOUS_MEAN

310.11999999999995

In [20]:
REWRITTEN_MEAN

11.303333333333333

In [21]:
ORIGINAL_MEAN

311.5933333333333

In [22]:
MALICIOUS_MEAN

1.4733333333333334

In [23]:
from scipy.stats import kendalltau
import numpy as np

taus = [kendalltau([x[0] for x in dict_sorted[k]], [x[0] for x in dict_sorted2[k]])[0] for k in dict_sorted]
print(f"Tau moyen: {np.mean(taus):.4f} ± {np.std(taus):.4f}")

ValueError: All inputs to `kendalltau` must be of the same size, found x-size 3220 and y-size 3218

In [ ]:
for key,value in dict_sorted.items():
    print(kendalltau(dict_sorted[key],dict_sorted2[key]))

ValueError: All inputs to `kendalltau` must be of the same size, found x-size 6440 and y-size 6436

In [25]:
#A COMPLETEMENT RECHECK
from scipy.stats import kendalltau
import numpy as np

# Convert dict_sorted and dict_sorted2 into proper ranking arrays for Kendall's Tau
taus = []
p_values = []

for key in dict_sorted:
    # Get the sorted lists for this author
    sorted1 = dict_sorted[key]
    sorted2 = dict_sorted2[key]
    
    # Create a mapping from paper_id to rank (position in sorted list)
    # Position 0 = rank 1 (highest similarity), position 1 = rank 2, etc.
    rank1 = {paper_id: rank for rank, (paper_id, score) in enumerate(sorted1)}
    rank2 = {paper_id: rank for rank, (paper_id, score) in enumerate(sorted2)}
    
    # Get common paper IDs and sort them to ensure consistent ordering
    common_papers = sorted(set(rank1.keys()) & set(rank2.keys()))
    
    # Create aligned ranking arrays: array1[i] = rank of paper i in dict_sorted
    #                                 array2[i] = rank of paper i in dict_sorted2
    array1 = [rank1[paper_id] for paper_id in common_papers]
    array2 = [rank2[paper_id] for paper_id in common_papers]
    
    # Compute Kendall's Tau
    tau, p_value = kendalltau(array1, array2)
    taus.append(tau)
    p_values.append(p_value)

print(f"Kendall's Tau moyen: {np.mean(taus):.4f} ± {np.std(taus):.4f}")
print(f"Min: {np.min(taus):.4f}, Max: {np.max(taus):.4f}")
print(f"P-value moyen: {np.mean(p_values):.2e}")


Kendall's Tau moyen: 0.4841 ± 0.0606
Min: 0.2812, Max: 0.6347
P-value moyen: 8.45e-129
